In [ ]:
# 导入库
import pandas as pd
import random  # 导入Python的random模块，用于生成随机数
import numpy as np  # 导入numpy模块，用于数组和矩阵运算
from scipy.interpolate import interp1d as inter
import matplotlib.pyplot as plt  # 导入matplotlib.pyplot模块，用于绘图
from matplotlib import rcParams
import copy
import scipy
import math
import warnings
from openpyxl import Workbook, load_workbook
import os

In [ ]:
data_SMX = pd.read_excel(r"D:\粒子群代码\粒子群代码\水库水位数据.xlsx", sheet_name="三门峡基本数据")
data_XLD = pd.read_excel(r"D:\粒子群代码\粒子群代码\水库水位数据.xlsx", sheet_name="小浪底基本数据")


In [ ]:

# 三门峡插值
H_SMX= data_SMX['H（m）'].dropna().values  # 水位数据
V_SMX = data_SMX['V（亿m3）'].dropna().values  # 对应的 V 值

H_cha_V_SMX = inter(H_SMX, V_SMX, kind='linear', bounds_error=False, fill_value=(V_SMX[0], V_SMX[-1]))
V_cha_H_SMX = inter(V_SMX, H_SMX, kind='linear', bounds_error=False, fill_value=(H_SMX[0], H_SMX[-1]))

Z_SMX_tail= data_SMX['尾水位（m）'].dropna().values  # 水位数据
Q_SMX_tail = data_SMX['下泄流量（m3/s）'].dropna().values  # 对应的 V 值

Z_cha_Q_SMX = inter(Z_SMX_tail, Q_SMX_tail, kind='linear', bounds_error=False, fill_value=(Q_SMX_tail[0], Q_SMX_tail[-1]))
Q_cha_Z_SMX = inter(Q_SMX_tail, Z_SMX_tail, kind='linear', bounds_error=False, fill_value=(Z_SMX_tail[0], Z_SMX_tail[-1]))

# 小浪底插值
H_XLD = data_XLD['H（m）'].dropna().values  # 水位数据
V_XLD = data_XLD['V（亿m3）'].dropna().values  # 对应的 V 值

H_cha_V_XLD = inter(H_XLD, V_XLD, kind='linear', bounds_error=False, fill_value=(V_XLD[0], V_XLD[-1]))
V_cha_H_XLD = inter(V_XLD, H_XLD, kind='linear', bounds_error=False, fill_value=(H_XLD[0], H_XLD[-1]))

Z_XLD_tail = data_XLD['尾水位（m）'].dropna().values  # 水位数据
Q_XLD_tail = data_XLD['下泄流量（m3/s）'].dropna().values  # 对应的 V 值

Z_cha_Q_XLD = inter(Z_XLD_tail, Q_XLD_tail, kind='linear', bounds_error=False, fill_value=(Q_XLD_tail[0], Q_XLD_tail[-1]))
Q_cha_Z_XLD = inter(Q_XLD_tail, Z_XLD_tail, kind='linear', bounds_error=False, fill_value=(Z_XLD_tail[0], Z_XLD_tail[-1]))



In [ ]:
#改
data_mod = pd.read_excel(r"D:\粒子群代码\粒子群代码\典型年.xlsx", sheet_name="平水年")

idex=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
idex_v=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,19]
idex_x_v=[ 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [ ]:

# 边界条件2022年
H_SMX = data_mod[data_mod['索引'] == '三门峡水位'][idex_v].dropna().values.flatten()
H_XLD =data_mod[data_mod['索引'] == '小浪底水位'][idex_v].dropna().values.flatten()

# 潼关流量和沙量
Q_in_SMX = data_mod[data_mod['索引'] == '潼关流量'][idex].dropna().values.flatten()
S_in_SMX = data_mod[data_mod['索引'] == '潼关含沙量'][idex].dropna().values.flatten()
In_list_SMX=np.concatenate((Q_in_SMX, S_in_SMX))

#水位条件
position_SMX_SD=[data_mod[data_mod['索引'] == '三门峡水位'][1].dropna().values.flatten()[0],data_mod[data_mod['索引'] == '三门峡水位'][19].dropna().values.flatten()[0]]
position_XLD_SD=[data_mod[data_mod['索引'] == '小浪底水位'][1].dropna().values.flatten()[0],data_mod[data_mod['索引'] == '小浪底水位'][19].dropna().values.flatten()[0]]


# 权重
W_ws_list=[0.29876519, 0.2723897 , 0.2516935 , 0.17715161]
W_eco_list=[0.33149767, 0.32268193, 0.13190986, 0.21391054]
W_sed_list=[ 0.1434174,0.41202221, 0.44463433, 0]
W_ele_list=[0.39381893, 0.3127165, 0.29346457]
W_list=W_ws_list+W_eco_list+W_sed_list+W_ele_list

time_list=np.array([3,3,3,3,1,1,1,1,1,1,1,1,1,3,3,3,3,3])
K=[8,7.5,8.3]
N=[440000,290000,1800000]
# 区间入流
Q_interval_list_XH=data_mod[data_mod['索引'] == '小花间'][idex].dropna().values.flatten()
Q_interval_list_HG=data_mod[data_mod['索引'] == '花高间'][idex].dropna().values.flatten()
Q_interval_list_GA=data_mod[data_mod['索引'] == '高艾间'][idex].dropna().values.flatten()
Q_interval_list_AL=data_mod[data_mod['索引'] == '艾利间'][idex].dropna().values.flatten()
interval_flow_list=np.vstack((Q_interval_list_XH, Q_interval_list_HG,Q_interval_list_GA,Q_interval_list_AL))

Coefficient=[[0.0000293486693244765,2.461270593,0,270.6218702],[0.705642338,1.215138284	,0.428884616,746.0168318],[2.464089335,1.053045706,0.353487002,-595.1485105],[1.751350379,1.068788759,0.580157647,-217.470081],[1.270366999,1.08552324,0.57886304	,-503.0271176],[1.337107021,0.991276171,0.904407635,14.99310904],[0.201316631,1.067315644,1.699244637,56.10852693,0.958257325],[2.268759297,0.914871725,0.934709566,-338.0942493]]

Q_life_list=[48.3522173489279]*18
Q_agr_list=data_mod[data_mod['索引'] == '农业供水变量上限'][idex].dropna().values.flatten()
Q_agr_particle_list=data_mod[data_mod['索引'] == '农业供水（初始）'][idex].dropna().values.flatten()
Z_list_CHU=np.concatenate((H_SMX,H_XLD,Q_agr_particle_list))
# print(Q_agr_list,Q_agr_particle_list)
eco_basic_flow=data_mod[data_mod['索引'] == '最小生态流量'][idex].dropna().values.flatten()
eco_suitable_flow=data_mod[data_mod['索引'] == '适宜生态流量'][idex].dropna().values.flatten()
eco_list=np.vstack((eco_basic_flow, eco_suitable_flow))

# 约束条件
# 小浪底水位
restraint_list_SMX=data_mod[data_mod['索引'] == '三门峡下泄流量约束（上限）'][idex].dropna().values.flatten()
restraint_list_XLD=data_mod[data_mod['索引'] == '小浪底下泄流量约束（上限）'][idex].dropna().values.flatten()
restraint_list_SMX_min=data_mod[data_mod['索引'] == '三门峡下泄流量约束（下限）'][idex].dropna().values.flatten()
restraint_list_XLD_min=data_mod[data_mod['索引'] == '小浪底下泄流量约束（下限）'][idex].dropna().values.flatten()
restraint_list_LJ=data_mod[data_mod['索引'] == '利津断面约束（下限）'][idex].dropna().values.flatten()
restraint_list = np.stack((restraint_list_SMX, restraint_list_SMX_min,restraint_list_XLD, restraint_list_XLD_min,restraint_list_LJ), axis=0)
# 速度
vel_inti_SMX=data_mod[data_mod['索引'] == '三门峡速度'][idex_x_v].dropna().values.flatten()
vel_inti_XLD=data_mod[data_mod['索引'] == '小浪底速度'][idex_x_v].dropna().values.flatten()
vel_inti_arg=data_mod[data_mod['索引'] == '农业供水速度'][idex].dropna().values.flatten()
# 耦合协调模型
Coupling_degree=0


In [ ]:
def Q_be(Q_out_list,Qs_list,time_list,K,Q_be_0):
    Q_x=0
    Qs_x=0
    Q=0
    Qs=0
    for i in range(18):
        if i in range(10,16):
            Q_x += Q_out_list[i] *time_list[i]/12
            Qs_x +=Qs_list[i]*time_list[i]/12
        else:
            Q+= Q_out_list[i] *time_list[i]/24
            Qs +=Qs_list[i]*time_list[i]/24
    lc_x=Qs_x/(Q_x**2)
    lc=Qs/(Q**2)
    Q_be_1=K[0]*(lc**K[3])*(2.71828**(K[1]*lc_x))*(Q_x**K[2])+K[4]
    Q_be=(1-2.71828**(-K[5]))*Q_be_1+Q_be_0*(2.71828**(-K[5]))
    return Q_be

In [ ]:
def store_data(data,name):
    for key, value in data.items():
        if isinstance(value, np.ndarray):  # 如果值是NumPy数组
            data[key] = value.tolist()  # 转换为普通列表
        if not isinstance(value, list):  # 如果值不是列表
            data[key] = [value]  # 转换为列表
    # 找出最长数据长度
    max_length = max(len(v) for v in data.values())

    # 将所有列表补全到相同长度
    for key, value in data.items():
        data[key] += [None] * (max_length - len(value))  # 用 None 补全

    # 转换为 DataFrame 并转置
    df = pd.DataFrame(data).T

    # 添加列名
    months = ["月份"] + ["1月", "2月", "3月", "4月", "5月", "5月", "5月", "6月", "6月", "6月", "7月", "7月", "7月", "8月", "9月", "10月", "11月", "12月", "13月"][:max_length]
    df.reset_index(inplace=True)
    df.columns = months  # 设置列名
    file_name = f"{name}.xlsx"
    # 保存到 Excel
    df.to_excel(file_name, index=False, engine="openpyxl")
    print(f"数据已成功保存到 {file_name}")

In [ ]:
#输沙
def allocate(Q_out_list,Qs_l):#分配沙量
    k_51 = Qs_l[4]*3*(Q_out_list[4]/sum(Q_out_list[4:7]))
    k_52 = Qs_l[4]*3*(Q_out_list[5]/sum(Q_out_list[4:7]))
    k_53 = Qs_l[4]*3*(Q_out_list[6]/sum(Q_out_list[4:7]))

    k_61= Qs_l[5]*3*(Q_out_list[7]/sum(Q_out_list[7:10]))
    k_62= Qs_l[5]*3*(Q_out_list[8]/sum(Q_out_list[7:10]))
    k_63= Qs_l[5]*3*(Q_out_list[9]/sum(Q_out_list[7:10]))

    k_71= Qs_l[6]*3*(Q_out_list[10]/sum(Q_out_list[10:13]))
    k_72= Qs_l[6]*3*(Q_out_list[11]/sum(Q_out_list[10:13]))
    k_73= Qs_l[6]*3*(Q_out_list[12]/sum(Q_out_list[10:13]))

    # Qs_list = Qs_l[:4] + [k_51] + [k_52] + [k_53] + [k_61] + [k_62] + [k_63] + [k_71] + [k_72] + [k_73] + Qs_l[7:]
    Qs_list =np.concatenate((Qs_l[:4], [k_51, k_52, k_53, k_61, k_62, k_63, k_71, k_72, k_73],Qs_l[7:]))
    return Qs_list

In [ ]:
#输沙
def sed_river_channel(Qs_l_up,Q_out_l_up,Q_out_l_present,C_x_present,C_fx_present):#河道输沙(已经验证)
    Qs_l_present=[]
    for period in range(12) :
        S=Qs_l_up[period]/Q_out_l_up[period]

        if Q_out_l_present[period] <=0:
            Q_out_present_t=0.00001
        else:
            Q_out_present_t=Q_out_l_present[period]
        if S <=0:
            S_t=0.00001
        else:
            S_t=S
#
        if period in range(6, 10):

            Qs_present_tem=C_x_present[0]*Q_out_present_t**C_x_present[1]*S_t**C_x_present[2]+C_x_present[3]

            if Qs_present_tem<=0:
                Qs_present=0.00001
            else :
                Qs_present=Qs_present_tem

        else:
            Qs_present_tem=C_fx_present[0]*Q_out_present_t**C_fx_present[1]*S_t**C_fx_present[2]+C_fx_present[3]
            if Qs_present_tem<=0:
                Qs_present=0.00001
            else :
                Qs_present=Qs_present_tem
# 储存值
        Qs_l_present.append(Qs_present)
    return Qs_l_present

In [ ]:
#输沙
def combine_flow(Q_out_l_1):#算出平均水量
    Q_out_l_1=Q_out_l_1
    average_Q_out_5 = sum(Q_out_l_1[4:7]) / len(Q_out_l_1[10:13])
    average_Q_out_6 = sum(Q_out_l_1[7:10]) / len(Q_out_l_1[10:13])
    average_Q_out_7 = sum(Q_out_l_1[10:13]) / len(Q_out_l_1[10:13])
    Q_out_l = np.concatenate((Q_out_l_1[:4], Q_out_l_1[13:]))

    Q_out_l = np.insert(Q_out_l, 4, average_Q_out_5)  # 在索引 4 插入 average_Q_out_5
    Q_out_l = np.insert(Q_out_l, 5, average_Q_out_6)  # 在索引 5 插入 average_Q_out_6
    Q_out_l = np.insert(Q_out_l, 6, average_Q_out_7)  # 在索引 6 插入 average_Q_out_7
    return Q_out_l

In [ ]:
#输沙

def sed_removal(S_list_TG,Q_out_list_SMX,Q_out_list_XLD,Z_list_XLD,Q_out_list_HYK,Q_out_list_GC,Q_out_list_AS,Q_out_list_LJ,Coefficient):

    Q_out_l_SMX_1 = Q_out_list_SMX.copy()
    S_l_TG_1=S_list_TG.copy()
    Q_out_l_XLD_1=Q_out_list_XLD.copy()
    Z_l_XLD_1=Z_list_XLD.copy()
    Q_out_l_HYK_1=Q_out_list_HYK.copy()
    Q_out_l_GC_1=Q_out_list_GC.copy()
    Q_out_l_AS_1=Q_out_list_AS.copy()
    Q_out_l_LJ_1=Q_out_list_LJ.copy()
    # 河道排沙系数
    C_fx_HYK=Coefficient[0]
    C_x_HYK=Coefficient[1]
    C_fx_GC=Coefficient[2]
    C_x_GC=Coefficient[3]
    C_fx_AS=Coefficient[4]
    C_x_AS=Coefficient[5]
    C_fx_LJ=Coefficient[6]
    C_x_LJ=Coefficient[7]
#初始化
    Qs_l_SMX=[]
    Qs_l_XLD=[]
# 对潼关的含沙量进行处理
    average_TG_5 = S_l_TG_1[4]
    average_TG_6 = S_l_TG_1[7]
    average_TG_7 = S_l_TG_1[10]
    S_l_TG =np.concatenate((S_l_TG_1[:4], S_l_TG_1[13:]))
    S_l_TG = np.insert(S_l_TG, 4, average_TG_5)  # 在索引 4 插入 average_TG_5
    S_l_TG = np.insert(S_l_TG, 5, average_TG_6)  # 在索引 5 插入 average_TG_6
    S_l_TG = np.insert(S_l_TG, 6, average_TG_7)  # 在索引 6 插入 average_TG_7
    # 对三门峡的下泄流量进行处理
    Q_out_l_SMX=combine_flow(Q_out_l_SMX_1)

    # 对小浪底的下泄流量进行处理
    Q_out_l_XLD=combine_flow(Q_out_l_XLD_1)

    # 对小浪底水位处理
    average_Z_l_XLD_5 = Z_l_XLD_1[4]
    average_Z_l_XLD_6 = Z_l_XLD_1[7]
    average_Z_l_XLD_7 = Z_l_XLD_1[10]
    Z_l_XLD = np.concatenate((Z_l_XLD_1[:4], Z_l_XLD_1[13:]))

    Z_l_XLD = np.insert(Z_l_XLD, 4, average_Z_l_XLD_5)
    # 在索引 5 插入 average_Z_l_XLD_6
    Z_l_XLD = np.insert(Z_l_XLD, 5, average_Z_l_XLD_6)
    # 在索引 6 插入 average_Z_l_XLD_7
    Z_l_XLD = np.insert(Z_l_XLD, 6, average_Z_l_XLD_7)
    # 对花园口的下泄流量进行处理
    Q_out_l_HYK=combine_flow(Q_out_l_HYK_1)

    # 对高村的下泄流量进行处理
    Q_out_l_GC=combine_flow(Q_out_l_GC_1)

   # 对艾山的下泄流量进行处理
    Q_out_l_AS=combine_flow(Q_out_l_AS_1)

    # 对利津的下泄流量进行处理
    Q_out_l_LJ=combine_flow(Q_out_l_LJ_1)

    for period in range(12) :
        if period in range(6, 10):
        #潼关沙量kg/m3
            S_tg=S_l_TG[period]
            if Q_out_l_SMX[period] > 0:
                Qs_SMX_tem=7.35431961566873*Q_out_l_SMX[period]**0.859192027311677*S_tg**0.832008017565135-3111.90679503533
                if Qs_SMX_tem <= 0 :
                    Qs_SMX_real=0.00001
                else:
                    Qs_SMX_real=Qs_SMX_tem

            else:
                Qs_SMX_real=0.00001
        else:
            Qs_SMX_real=0.00001
# 储存值
        Qs_l_SMX.append(Qs_SMX_real)

# 计算小浪底输沙
    for period in range(12) :
        if period in range(6, 10):
            if Q_out_l_SMX[period]<= 0 or Qs_l_SMX[period]<= 0 or Q_out_l_XLD[period]<= 0 :
                Qs_XLD=0.00001
            else:
                Qs_tem_XLD=0.0000963754804283109*(Q_out_l_XLD[period]**10.9533234224183*(275-((Z_l_XLD[period]+Z_l_XLD[period+1])/2))**1.46466294485378/Q_out_l_SMX[period]**6.92266750906981/Qs_l_SMX[period]**(-2.45570964584068)/128000**1.2)**0.402418973484749

                if Qs_tem_XLD >0:
                    if Qs_tem_XLD/Q_out_l_XLD[period]<60:

                        Qs_XLD=Qs_tem_XLD
                    else:
                        Qs_XLD=Q_out_l_XLD[period]*60

                else:
                    Qs_XLD=0.00001
        else:
            Qs_XLD=0.00001
# 储存值
        Qs_l_XLD.append(Qs_XLD)

# 花园口输沙
    Qs_l_HYK=sed_river_channel(Qs_l_XLD,Q_out_l_XLD,Q_out_l_HYK,C_x_HYK,C_fx_HYK)


# 高村输沙
    Qs_l_GC=sed_river_channel(Qs_l_HYK,Q_out_l_HYK,Q_out_l_GC,C_x_GC,C_fx_GC)

# # 艾山输沙
    Qs_l_AS=sed_river_channel(Qs_l_GC,Q_out_l_GC,Q_out_l_AS,C_x_AS,C_fx_AS)

# # 利津输沙
    Qs_l_LJ=sed_river_channel(Qs_l_AS,Q_out_l_AS,Q_out_l_LJ,C_x_LJ,C_fx_LJ)

    Qs_list_SMX=allocate(Q_out_list_SMX,Qs_l_SMX)
    Qs_list_XLD=allocate(Q_out_list_XLD,Qs_l_XLD)
    Qs_list_GC=allocate(Q_out_list_GC,Qs_l_GC)
    Qs_list_AS=allocate(Q_out_list_AS,Qs_l_AS)
    Qs_list_HYK=allocate(Q_out_list_HYK,Qs_l_HYK)
    Qs_list_LJ=allocate(Q_out_list_LJ,Qs_l_LJ)
    return Q_out_l_SMX,Q_out_l_XLD,Q_out_l_HYK,Q_out_l_GC,Q_out_l_AS,Q_out_l_LJ,Qs_l_SMX,Qs_l_XLD,Qs_l_HYK,Qs_l_GC,Qs_l_AS,Qs_l_LJ,Qs_list_SMX,Qs_list_XLD,Qs_list_HYK,Qs_list_GC,Qs_list_AS,Qs_list_LJ


In [ ]:
def geometric_mean(values):
    values = np.array(values)
    if np.any(values <= 0):
        return 0
    return np.exp(np.mean(np.log(values)))

In [ ]:
# 目标函数是根据第五改的
def objective_function(Z_list,W_list, In_list_SMX,time_list,K,N,interval_flow_list,Coefficient,Q_life_list,Q_agr_list,eco_list,restraint_list):
# 变量
    Z_list_SMX = Z_list[:19]
    Z_list_XLD = Z_list[19:38]
    Q_agr_particle_list= Z_list[38:57]
# 权重
    W_ws_list=W_list[0:4]
    W_eco_list=W_list[4:8]
    W_sed_list=W_list[8:12]
    W_ele_list=W_list[12:15]
# 边界条件
    Q_list_TG=In_list_SMX[:18]
    S_list_TG=In_list_SMX[18:]
# 时间
    Time_list=[]
    for i in range(len(time_list)):
        T_period=time_list[i]*365/36*24*60*60
        Time_list.append(T_period)
# 发电
#     出力系数
    K_fx_SMX=K[0]
    K_x_SMX=K[1]
    K_XLD=K[2]
#    出力上限
    N_max_fx_SMX=N[0]
    N_max_x_SMX=N[1]
    N_max_XLD=N[2]
#排沙
    Coefficient=Coefficient
# 非农业需水量
#     Q_life_list
# 农业需水量
#     Q_agr_list
# 生态需水量
    eco_basic_flow=eco_list[0]
    eco_suitable_flow=eco_list[1]
# 协调耦合模型
#     Coupling_degree
# 约束
#     restraint_list

# 初始化

# 水位
    # 三门峡
    Z_up_list_SMX=[]
    Z_down_list_SMX=[]
    delta_H_list_SMX=[]
    # 小浪底
    H_list_XLD       = []
    Z_up_list_XLD    = []
    Z_down_list_XLD  = []
# 水量
    # 三门峡
    Q_out_list_SMX=[]
    # 小浪底
    Q_out_list_XLD   = []
    Q_in_list_XLD    = []
    # 花园口
    Q_out_list_HYK=[]
    # 高村
    Q_out_list_GC=[]
    # 艾山
    Q_out_list_AS=[]
    # 利津
    Q_out_list_LJ=[]
# 发电量
    # 三门峡
    N_real_list_SMX = []
    # 小浪底
    P_ele_XLD_list=[0]*18  #保证率
    N_real_list_XLD  = []
# 沙量
    # 潼关
    Qs_list_TG  =[]
    WS_list_TG =[]
    # 三门峡
    Qs_list_SMX  =[]
    # 小浪底
    Qs_list_XLD  = []
    WS_list_XLD  =[]
    # 花园口
    Qs_list_HYK   =[]
    # 高村
    Qs_list_GC   =[]
    # 艾山
    Qs_list_AS   =[]
    # 利津
    Qs_list_LJ   =[]
    WS_list_LJ  =[]
# XLD调水调沙时长
    Qs_time_list_HYK=[]#单位天
    Q_wsr_list=[]#单位m3/s
    Q_sed_wsr=0
# 生态
    P_eco_list=[]#最小生态基流保证率
    ECO_suitbale_shortage_depth_list=[]
    ECO_suitbale_shortage_list=[]
    ECO_suitbale_list=[]
    Q_eco_list = []
    ECO_pulse_list  =[]
# 供水
    P_life_list=[]              # 总满足生活工业供水
    V_agr_shortage_list=[]
    V_agr_shortage_depth_list=[]
# 计算

    for period in range(18):

        Z_begin_SMX = Z_list_SMX[period]
        Z_end_SMX = Z_list_SMX[period + 1]
        Q_in_SMX = Q_list_TG[period]


#计算三门峡流量
        Q_out_SMX = Q_in_SMX - (H_cha_V_SMX(Z_end_SMX) - H_cha_V_SMX(Z_begin_SMX)) * 10 ** 8 / Time_list[period]
        Z_up_SMX = (Z_begin_SMX + Z_end_SMX) / 2
        Z_down_SMX = Q_cha_Z_SMX(Q_out_SMX)
        delta_H_SMX = Z_up_SMX - Z_down_SMX
# 储存值
        Q_out_list_SMX.append(Q_out_SMX)
        Z_up_list_SMX.append(Z_up_SMX)
        Z_down_list_SMX.append(Z_down_SMX)
        delta_H_list_SMX.append(delta_H_SMX)

# 计算三门峡出力
        # 给k和n赋值
        if period in range(10,16):
            K_SMX=K[1]
            N_max_SMX=N[1]
        else:
            K_SMX=K[0]
            N_max_SMX=N[0]
        # 计算算暂时出力
        N_temporary_SMX = K_SMX * Q_out_SMX * delta_H_SMX

        if N_temporary_SMX <= 0 or Q_out_SMX <= 0:
            N_real_SMX = 0
        elif N_temporary_SMX >= N_max_SMX:
            N_real_SMX = N_max_SMX
        else:
            N_real_SMX = N_temporary_SMX
# 储存值
        N_real_list_SMX.append(N_real_SMX)



# 小浪底
    for period in range(18):  # XLD水量

# 小浪底的水位及流量
        Z_begin_XLD = Z_list_XLD[period]
        Z_end_XLD = Z_list_XLD[period + 1]
        Q_in_XLD = Q_out_list_SMX[period] #+ Q_interval[period]  # 三门峡的出流加上中间的入流
# 储存值
        Q_in_list_XLD.append(Q_in_XLD)
# 计算小浪底流量
        Q_out_XLD = Q_in_XLD - (H_cha_V_XLD(Z_end_XLD) - H_cha_V_XLD(Z_begin_XLD)) * 10 ** 8 / Time_list[period]
        Z_up_XLD = (Z_begin_XLD + Z_end_XLD) / 2
        Z_down_XLD = Q_cha_Z_XLD(Q_out_XLD)
        delta_H_XLD = Z_up_XLD - Z_down_XLD
# 储存值
        H_list_XLD.append(delta_H_XLD)
        Q_out_list_XLD.append(Q_out_XLD)
        Z_up_list_XLD.append(Z_up_XLD)
        Z_down_list_XLD.append(Z_down_XLD)
# 计算小浪底出力
        N_temporary_XLD = K_XLD * Q_out_XLD * delta_H_XLD

        if N_temporary_XLD <= 0:
            N_real_XLD = 0.00001
        elif N_temporary_XLD >= N_max_XLD:
            N_real_XLD = N_max_XLD
        else:
            N_real_XLD = N_temporary_XLD
# 储存值
        N_real_list_XLD.append(N_real_XLD)


# 花园口流量
        Q_out_HYK=Q_out_XLD-interval_flow_list[0][period]
# 储存值
        Q_out_list_HYK.append(Q_out_HYK)
# 高村流量
        Q_out_GC=Q_out_HYK-interval_flow_list[1][period]-(Q_agr_particle_list[period]*0.277533019252517)
# 储存值
        Q_out_list_GC.append(Q_out_GC)
# 艾山流量
        Q_out_AS=Q_out_GC-interval_flow_list[2][period]-(Q_agr_particle_list[period]*0.203117977080216)
# 储存值
        Q_out_list_AS.append(Q_out_AS)
# 利津流量
        Q_out_LJ=Q_out_AS-interval_flow_list[3][period]-(Q_agr_particle_list[period]*0.467779820160666)
# 储存值
        Q_out_list_LJ.append(Q_out_LJ)

# 调水调沙（以花园口为控制断面）（时长，流量）
        if period in range(9,11):
            Q_wsr=Q_out_HYK
            if Q_out_HYK > 4000:
                Qs_time_HYK=10
            elif Q_out_HYK >= 2400:
                Qs_time_HYK=10
            else:
                Qs_time_HYK=0

        else:
            Qs_time_HYK=0
            Q_wsr=0

# 储存值
        Q_wsr_list.append(Q_wsr)
        Qs_time_list_HYK.append(Qs_time_HYK)
# 非农业供水
        Q_life = Q_life_list [period]
        if Q_life>=Q_life_list [period] :
            P_life=1
        else:
            P_life=0
# 储存值
        P_life_list.append(P_life)

 # 农业供水
#缺水深度、缺水量（#亿立方米）
        Q_agr=Q_agr_list[period]
        if Q_out_HYK - Q_agr_particle_list[period]- Q_life>=0:
            Q_agr_particle=Q_agr_particle_list[period]
            if Q_agr_particle >= Q_agr:
                V_agr_shortage=0
                V_agr_shortage_depth=0
            else:
                V_agr_shortage= ((Q_agr-Q_agr_particle)*Time_list[period])/10**8#亿立方米
                V_agr_shortage_depth=1-(Q_agr_particle/Q_agr)
        else:
            V_agr_shortage= (Q_agr*Time_list[period])/10**8#亿立方米
            V_agr_shortage_depth=1
# 储存值
        V_agr_shortage_list.append(V_agr_shortage)
        V_agr_shortage_depth_list.append(V_agr_shortage_depth)

#生态流量
        Q_eco = Q_out_HYK - Q_agr_particle_list[period] - Q_life_list[period]
        Q_eco_list.append(Q_eco)

# 生态脉冲（次数）
        if period in range(4,10):
            if Q_out_HYK >= 1400 :
                ECO_pulse=1
            else:
                ECO_pulse=0
        else:
            ECO_pulse=0
# 储存值
        ECO_pulse_list.append(ECO_pulse)
#最小基流保证率
        if Q_eco < eco_basic_flow[period]:
            P_eco=0
        else:
            P_eco=1
# 储存值
        P_eco_list.append(P_eco)
#适宜生态基流缺水深度

        if Q_eco < eco_suitable_flow[period]:    # 如果生态供水量 小于 生态适宜流量
            ECO_suitable_shortage_depth_time = ((eco_suitable_flow[period] -eco_basic_flow[period])- (Q_eco -eco_basic_flow[period]) )/ (eco_suitable_flow[period]-eco_basic_flow[period])
            if ECO_suitable_shortage_depth_time > 1:
               ECO_suitable_shortage_depth = 1
            else:
                ECO_suitable_shortage_depth=ECO_suitable_shortage_depth_time
        else:
            ECO_suitable_shortage_depth = 0
# 储存值
        ECO_suitbale_shortage_depth_list.append(ECO_suitable_shortage_depth)
#适宜生态基流缺水量

        if Q_eco < eco_suitable_flow[period]:    # 如果生态供水量 小于 生态适宜流量
            ECO_suitable_shortage = (((eco_suitable_flow[period] -eco_basic_flow[period])- (Q_eco -eco_basic_flow[period]) )*Time_list[period])/10**8#亿立方米

        else:
            ECO_suitable_shortage = 0

# 储存值
        ECO_suitbale_shortage_list.append(ECO_suitable_shortage)
#适宜生态基流-基本生态流量

        ECO_suitable=((eco_suitable_flow[period]-eco_basic_flow[period])*Time_list[period])/10**8#亿立方米

        ECO_suitbale_list.append(ECO_suitable)
# 各个断面排沙计算
    Q_out_l_SMX,Q_out_l_XLD,Q_out_l_HYK,Q_out_l_GC,Q_out_l_AS,Q_out_l_LJ,Qs_l_SMX,Qs_l_XLD,Qs_l_HYK,Qs_l_GC,Qs_l_AS,Qs_l_LJ,Qs_list_SMX,Qs_list_XLD,Qs_list_HYK,Qs_list_GC,Qs_list_AS,Qs_list_LJ=sed_removal(S_list_TG,Q_out_list_SMX,Q_out_list_XLD,Z_list_XLD,Q_out_list_HYK,Q_out_list_GC,Q_out_list_AS,Q_out_list_LJ,Coefficient)

# 各个子目标求解

# 供水子目标（关键时期为）（五月上旬到六月上旬）（七月下旬到八月下旬）
    WS1=0#（缺水量）
    WS2=0#（缺水量）
    MDR1=0
# 需要的水量
    W1=0
    W2=0
# 生活供水
    Plif=sum(P_life_list)/18
# 农业缺水
    for period in range(18):
        if period in range(4,8) or period in range(12,14):#关键时期 农业关键期的总缺水量 WS1
            WS1+=V_agr_shortage_list[period]
            W1+=(Q_agr_list[period]*Time_list[period]) / 10**8

        else :                                              #非关键时期 农业关键期的总缺水量 WS2
            WS2+=V_agr_shortage_list[period]
            W2+=(Q_agr_list[period]*Time_list[period]) / 10**8
    MDR1 = max(max(V_agr_shortage_depth_list[4:8]), max(V_agr_shortage_depth_list[12:14]))
# 权重
    W_ws1=W_ws_list[0]
    W_ws2=W_ws_list[1]
    W_ws3=W_ws_list[2]
    W_ws4=W_ws_list[3]
# 供水子目标
    G_ws=W_ws1*Plif+W_ws2*(1-MDR1)+W_ws3*(1-WS1/W1)+W_ws4*(1-WS2/W2)
# 生态子目标（关键时期为）（五月、六月份）
    ESO_sum_pulse=0
    Feco1=0
    Feco2=0
    Peco=0
    MDR_eco=0
    V_shortage_eco=0
# 生态脉冲
    ESO_sum_pulse=sum(ECO_pulse_list)
    if ESO_sum_pulse == 0:
        Feco1=0
        Feco2=0
    elif ESO_sum_pulse == 1:
        Feco1=1
        Feco2=0
    elif ESO_sum_pulse  >  1:
        Feco1=1
        Feco2=(ESO_sum_pulse-1)/5
    else:
        Feco1=0
        Feco2=0
# 保证率、缺水深度
    Peco=(sum(P_eco_list*time_list))/36
    MDR_eco=max(ECO_suitbale_shortage_depth_list[4:10])
    V_shortage_eco=sum(ECO_suitbale_shortage_list)/sum(ECO_suitbale_list)
# 权重
    W_eco1=W_eco_list[0]
    W_eco2=W_eco_list[1]
    W_eco3=W_eco_list[2]
    W_eco4=W_eco_list[3]
# 生态子目标
    G_eco=W_eco1*Peco+W_eco2*Feco1+W_eco3*Feco2+W_eco4*(1-V_shortage_eco)
# 输沙子目标（6月下旬到7月上旬）
    T_sed=0
    Q_sed_be=0
# 水库排沙量（序参量）
    SED_sil_S_X=0#归一化
    SED_S_X=0
    WIGHT_S_X_r=0
    WIGHT_sil_S_X_r=0
    SED_sil_X_L=0#归一化
    WIGHT_X_L_c=0
    WIGHT_sil_X_L_c=0
    SED_X_L=0
#  潼关来沙量（小写的都被append列表里面了）
    qs_TG=0
    ws_tg=0
#  小浪底来沙量
    ws_xld=0
#  利津来沙量
    ws_lj=0
# 调水调沙
    if sum(Qs_time_list_HYK) >0:
        T_sed=sum(Qs_time_list_HYK)/20
    else:
        T_sed=0
    Q_sed_wsr=((max(0, min(Q_wsr_list[9], 4000)) / 4000) + (max(0, min(Q_wsr_list[10], 4000)) / 4000)) / 2
# 水库减淤量 （排沙量/潼关来沙量）
    for i in range(18):
        if S_list_TG[i]==0:
            S_in_SMX=0.001
        else:
            S_in_SMX= S_list_TG[i]
        qs_TG=S_in_SMX* Q_list_TG[i]

        ws_tg= (S_in_SMX* Q_list_TG[i] *Time_list[i] )/10**11#潼关来沙量kg
# 储存值
        Qs_list_TG.append(qs_TG)
        WS_list_TG.append(ws_tg)


    for i in range(18):
        if Qs_list_XLD[i]<=0:
            qs_XLD=0.0001
        else:
            qs_XLD=Qs_list_XLD[i]
        ws_xld= (qs_XLD *Time_list[i] )/10**11#潼关来沙量kg
# 储存值
        WS_list_XLD.append(ws_xld)
#水库排沙量
    WIGHT_S_X_r=sum(WS_list_XLD)
    WIGHT_sil_S_X_r=sum(WS_list_XLD)/6
#水库减淤量
    SED_S_X=sum(WS_list_TG)-sum(WS_list_XLD)
    SED_sil_S_X=1-((sum(WS_list_TG)-sum(WS_list_XLD)+3.77)/(3.77*2))
#河道减淤量

    for i in range(18):
        if Qs_list_LJ[i]==0:
            qs_lj=0.000001
        else:
            qs_lj=Qs_list_LJ[i]
        ws_lj= (qs_lj*Time_list[i] )/10**11#潼关来沙量kg
# 储存值
        WS_list_LJ.append(ws_lj)
#河道排沙量
    WIGHT_X_L_c=sum(WS_list_LJ)
    WIGHT_sil_X_L_c=sum(WS_list_LJ)/5
#河道减淤量
    SED_X_L=sum(WS_list_XLD)-sum(WS_list_LJ)
    SED_sil_X_L=1-((sum(WS_list_XLD)-sum(WS_list_LJ)+3)/(3*2))
# 平滩流量模块
# 参数
    K_HYK=[384.36, -41.85, 0.304, -0.063, 2542.03, 0.393]
    K_GC=[17220.69,-113.53, 0.362, 0.439, 1482.6, 0.215]
    K_AS=[104.58, 0, 0.482, -0.083, -741.58, 0.206]
    K_LJ=[108.83, -39.68, 0.711, 0.228, 2280.03, 0.2]

# 花园口平滩流量
    Q_be_0_HYK=7300
    Q_be_HYK=Q_be(Q_out_list_HYK,Qs_list_HYK,time_list,K_HYK,Q_be_0_HYK)
# 高村平滩流量
    Q_be_0_GC=6600
    Q_be_GC=Q_be(Q_out_list_GC,Qs_list_GC,time_list,K_GC,Q_be_0_GC)
# 艾山平滩流量
    Q_be_0_AS=4800
    Q_be_AS=Q_be(Q_out_list_AS,Qs_list_AS,time_list,K_AS,Q_be_0_AS)
# 利津平滩流量
    Q_be_0_LJ=4700
    Q_be_LJ=Q_be(Q_out_list_LJ,Qs_list_LJ,time_list,K_LJ,Q_be_0_LJ)

    min_Q_be = min(Q_be_HYK, Q_be_GC, Q_be_AS, Q_be_LJ)
    Q_be_list=[Q_be_HYK, Q_be_GC, Q_be_AS, Q_be_LJ]
#确保平滩流量大于4000
    if min_Q_be <= 5000:
        Q_sed_be= (min_Q_be-3000)/2000
    else:
        Q_sed_be=1

# 排沙子目标权重
    W_sed1=W_sed_list[0]
    W_sed2=W_sed_list[1]
    W_sed3=W_sed_list[2]
    W_sed4=W_sed_list[3]

# 输沙子目标
#
    G_sed= W_sed1*Q_sed_wsr+W_sed2*WIGHT_sil_S_X_r+W_sed3*WIGHT_sil_X_L_c
    G_sed_1= W_sed1*Q_sed_wsr+W_sed2*WIGHT_sil_S_X_r+W_sed3*WIGHT_sil_X_L_c
# 发电子目标（非枯水期）（4月-10月份）

# 初始化
    P_ele_XLD=0
    N_ele_key_XLD=0
    N_ele_key_XLD_s=0
    N_ele_nkey_XLD=0
    N_ele_nkey_XLD_s=0
    N_ele_XLD=0
    N_ele_XLD_s=0
    N_ele_SMX=0
    N_ele_SMX_s=0
# 发电子目标权重
    W_ele1=W_ele_list[0]
    W_ele2=W_ele_list[1]
    W_ele3=W_ele_list[2]
    # W_ele4=W_ele_list[3]

#非枯水期（4~10月）汛期小浪底发电保证率Pele
# 判断索引3到15的值是否大于305000，如果大于，则对应位置赋值为1
    P_ele_XLD_list = (np.array(N_real_list_XLD )>= 305000).astype(int)
    P_ele_XLD= (np.sum(np.array(N_real_list_XLD[3:16]) >= 305000))/13
#非枯水期（4~10月）汛期小浪底发电量Nele1
    N_max_list_XLD= np.array([1800000]*18)

    N_ele_key_XLD_s=np.sum(N_real_list_XLD[3:16]*time_list[3:16])

    N_ele_key_XLD=(np.sum(N_real_list_XLD[3:16]*time_list[3:16]))/(np.sum(N_max_list_XLD[3:16]*time_list[3:16]))
# 枯水期（11~3月）非汛期小浪底发电量Nele2
    N_ele_nkey_XLD_s=np.sum(N_real_list_XLD[0:3]*time_list[0:3])+np.sum(N_real_list_XLD[16:18]*time_list[16:18])

    N_ele_nkey_XLD=(np.sum(N_real_list_XLD[0:3]*time_list[0:3])+np.sum(N_real_list_XLD[16:18]*time_list[16:18]))/(np.sum(N_max_list_XLD[0:3]*time_list[0:3])+np.sum(N_max_list_XLD[16:18]*time_list[16:18]))
#小浪底发电量
    N_ele_XLD_s=np.sum(N_real_list_XLD[3:16]*time_list[3:16])+np.sum(N_real_list_XLD[0:3]*time_list[0:3])+np.sum(N_real_list_XLD[16:18]*time_list[16:18])
    N_ele_XLD=(np.sum(N_real_list_XLD[3:16]*time_list[3:16])+np.sum(N_real_list_XLD[0:3]*time_list[0:3])+np.sum(N_real_list_XLD[16:18]*time_list[16:18]))/(np.sum(N_max_list_XLD[3:16]*time_list[3:16])+np.sum(N_max_list_XLD[0:3]*time_list[0:3])+np.sum(N_max_list_XLD[16:18]*time_list[16:18]))

# 三门峡发电量Nele3
#三门峡的出力约束
    N_max_list_SMX = np.concatenate([
        np.full(10, 440000),  # 前10个元素除以440000
        np.full(6, 290000),   # 第11到第16个元素除以290000
        np.full(2, 440000)    # 第17和第18个元素除以440000
    ])
    N_ele_SMX_s=np.sum(N_real_list_SMX)
    N_ele_SMX=(np.sum(N_real_list_SMX*time_list))/(np.sum(N_max_list_SMX*time_list))
    # G_ele=W_ele1*P_ele_XLD+W_ele2*N_ele_key_XLD+W_ele3*N_ele_nkey_XLD+W_ele4*N_ele_SMX
    G_ele=W_ele1*P_ele_XLD+W_ele2*N_ele_XLD+W_ele3*N_ele_SMX
#总的协同度
    # values=[G_ws,G_eco,G_sed]
    # Coupling_degree(values,a=1/3,b=1/3,c=1/3)
    # D =Coupling_degree(values,a=1/3,b=1/3,c=1/3)
    # Coupling_degree(values)
    # D =Coupling_degree(values)
    # D=G_ws+G_eco+G_sed+G_ele
    D=geometric_mean([G_ws,G_eco,G_sed,G_ele])
    D_1=geometric_mean([G_ws,G_eco,G_sed_1,G_ele])
# 非负数约束
    for x in (Q_out_list_LJ,Q_out_list_SMX, Q_out_list_XLD,Q_out_list_AS,Q_out_list_HYK,Q_out_list_GC,Q_eco_list):
        for i in x:
            if i < 0  :
                D += i *0.0001

# 三门峡出流
# 上限

    for i in range(len(Q_out_list_SMX)):
        if Q_out_list_SMX[i] >restraint_list[0][i]:
            D += (restraint_list[0][i]-Q_out_list_SMX[i])/1000

# 下限

    for i in range(len(Q_out_list_SMX)):
        if Q_out_list_SMX[i] <0:
            D += (Q_out_list_SMX[i]-0)/1000
# 小浪底下泄流量
# 上限
    for i in range(len(Q_out_list_XLD)):
        if Q_out_list_XLD[i] >restraint_list[2][i]:
            D += (restraint_list[2][i]-Q_out_list_XLD[i])/1000
# 下限
    for i in range(len(Q_out_list_XLD)):
        if Q_out_list_XLD[i] < 0:
            D += (Q_out_list_XLD[i]-0)/1000

# 上限（利津断面约束）
    for i in range(len(Q_out_list_LJ)):
        if Q_out_list_LJ[i] -restraint_list[4][i] < 0:
            D += (Q_out_list_LJ[i]-restraint_list[4][i])/1000

# 下限（利津断面约束）
    for i in Q_out_list_LJ:
        if i  < 75:
            D -= (75-i)*0.001

    for x in (Q_out_list_AS,Q_out_list_HYK,Q_out_list_GC):
        for i in x:
            if i > 4000:
                D -= (i-4000)*0.0001

    for x in V_agr_shortage_depth_list:
        if x > 0.5:
            D -= (x-0.5)* 0.1

    #确保平滩流量大于4000
    if min_Q_be < 4000:
        D+= (min_Q_be-4000)/4000*0.00008

# 返回值
    data={

        "Z_up_list_SMX":Z_up_list_SMX,
        "Z_down_list_SMX":Z_down_list_SMX,
        "Z_up_list_XLD":Z_up_list_XLD,
        "Z_down_list_XLD":Z_down_list_XLD,

        "Q_list_TG":list(Q_list_TG),    #m3/s
        "Q_out_list_SMX":list(Q_out_list_SMX),
        "Q_out_list_XLD":list(Q_out_list_XLD),
        "Q_out_list_HYK":list(Q_out_list_HYK),
        "Q_out_list_GC":list(Q_out_list_GC),
        "Q_out_list_AS":list(Q_out_list_AS),
        "Q_out_list_LJ":list(Q_out_list_LJ),

        "Q_out_l_SMX": list(Q_out_l_SMX),
        "Q_out_l_XLD": list(Q_out_l_XLD),
        "Q_out_l_HYK": list(Q_out_l_HYK),
        "Q_out_l_GC": list(Q_out_l_GC),
        "Q_out_l_AS": list(Q_out_l_AS),
        "Q_out_l_LJ": list(Q_out_l_LJ),

        "S_list_TG ":list(S_list_TG),    #（kg/m3）
        "Qs_list_TG":list(Qs_list_TG),
        "Qs_list_SMX":list(Qs_list_SMX),
        "Qs_list_XLD":list(Qs_list_XLD),
        "Qs_list_HYK":list(Qs_list_HYK),
        "Qs_list_GC":list(Qs_list_GC),
        "Qs_list_AS":list(Qs_list_AS),
        "Qs_list_LJ":list(Qs_list_LJ),

        "Qs_l_SMX": list(Qs_l_SMX),
        "Qs_l_XLD": list(Qs_l_XLD),
        "Qs_l_HYK": list(Qs_l_HYK),
        "Qs_l_GC": list(Qs_l_GC),
        "Qs_l_AS": list(Qs_l_AS),
        "Qs_l_LJ": list(Qs_l_LJ),

        "delta_H_list_SMX":delta_H_list_SMX,
        "H_list_XLD":H_list_XLD,


        # 供水
        "P_life_list":P_life_list,    #生活供水保证率
        "V_agr_shortage_list":V_agr_shortage_list,#单位亿立方米
        "V_agr_shortage_depth_list":V_agr_shortage_depth_list,
        "Plif":Plif,
        "WS1":WS1,#亿立方米
        "W1":W1,
        "MDR1":MDR1,
        "WS2":WS2,
        "W2":W2,
        "G_ws":G_ws,
        #生态供水
        "Q_eco_list":Q_eco_list,
        "P_eco_list":P_eco_list,
        "Peco":Peco,
        "ECO_pulse_list":ECO_pulse_list,
        "Feco1":Feco1,
        "ECO_suitbale_shortage_depth_list":ECO_suitbale_shortage_depth_list,
        "MDR_eco":MDR_eco,
        "ECO_suitbale_shortage_list":ECO_suitbale_shortage_list,
        "ECO_suitbale_list":ECO_suitbale_list,
        "V_shortage_eco":V_shortage_eco,
        "Feco2":Feco2,
        "G_eco":G_eco,
        #排沙
        "Q_sed_wsr":Q_sed_wsr,
        "Q_wsr_list": list(Q_wsr_list),
        "Q_sed_be":[Q_sed_be],
        "Qs_time_list_HYK" : Qs_time_list_HYK,
        "T_sed":T_sed,
        "WS_list_TG":WS_list_TG,
        "WS_list_XLD":WS_list_XLD,
        "SED_S_X":SED_S_X,
        "SED_sil_S_X":SED_sil_S_X,#归一化
        "WIGHT_S_X_r":WIGHT_S_X_r,
        "WIGHT_sil_S_X_r":WIGHT_sil_S_X_r,
        "WS_list_LJ":WS_list_LJ,
        "SED_X_L":SED_X_L,
        "SED_sil_X_L":SED_sil_X_L,
        "WIGHT_X_L_c":WIGHT_X_L_c,
        "WIGHT_sil_X_L_c":WIGHT_sil_X_L_c,
        "G_sed":G_sed,
        "G_sed_1":G_sed_1,
        #发电
        "N_real_list_SMX":list(N_real_list_SMX),
        "N_ele_SMX_s":N_ele_SMX_s,
        "N_ele_SMX":N_ele_SMX,
        "N_real_list_XLD": N_real_list_XLD,
        "P_ele_XLD_list":list(P_ele_XLD_list),
        "P_ele_XLD":P_ele_XLD,
        "N_ele_key_XLD_s":N_ele_key_XLD_s,
        "N_ele_key_XLD":N_ele_key_XLD,
        "N_ele_nkey_XLD_s":N_ele_nkey_XLD_s,
        "N_ele_nkey_XLD":N_ele_nkey_XLD,
        "N_ele_XLD _s":N_ele_XLD_s,
        "N_ele_XLD":N_ele_XLD,
        "G_ele":G_ele,
        "D":D,
        "D_1":D_1,
        "Q_be_list":Q_be_list
    }

    return {"data":data,"D":D}



In [ ]:
# 第六改
class SAPSO:
    # SAPSO类的构造函数，初始化一些参数
    def __init__(self, objective_function, num_particles, num_iterations, initial_temp, final_temp,position_SMX_SD,position_XLD_SD,W_list, In_list_SMX,time_list,K, N,interval_flow_list,Coefficient,Q_life_list,Q_agr_list,eco_list,restraint_list,initial_best_position,initial_best_value):
        self.objective_function = objective_function  # 目标函数，我们要优化的函数
        self.num_particles = num_particles  # 粒子的数量
        self.num_iterations = num_iterations  # 迭代的次数
        self.initial_temp = initial_temp  # 初始温度
        self.final_temp = final_temp  # 最终温度
        self.position_SMX_SD=position_SMX_SD
        self.position_XLD_SD=position_XLD_SD
        self.W_list=W_list# 权重
        self.In_list_SMX = In_list_SMX
        self.time_list=time_list
        self.N= N
        self.K = K
        self.interval_flow_list=interval_flow_list
        self.Coefficient=Coefficient
        self.Q_life_list=Q_life_list
        self.Q_agr_list=Q_agr_list
        self.eco_list=eco_list
        self.restraint_list=restraint_list

        self.global_best_position = np.array(initial_best_position)  # 全局最优位置
        self.global_best_value = initial_best_value  # 全局最优值

        self.global_best_values = []  # 保存每次迭代的全局最优值


    def initialize_particles(self):
        particles = []  #初始化列表以储存粒子
        for _ in range(self.num_particles):  # 对每个粒子进行赋值

            # 生成三门峡水位
            position_SMX=[]
            for _ in range(10):
                position_SMX.append(random.uniform(310, 318))
            for _ in range(4):
                position_SMX.append(random.uniform(300, 305))
            for _ in range(3):
                position_SMX.append(random.uniform(300, 318))
            position_SMX = np.insert(position_SMX, 0, self.position_SMX_SD[0])  # 水库的初始水位和最终水位已经确定
            position_SMX = np.append(position_SMX, self.position_SMX_SD[1])
            # 生成小浪底水位
            position_XLD=[]
            # 2月到6.20
            for _ in range(9):
                position_XLD.append(random.uniform(215, 275))
            # 7.1到8.31
            for _ in range(5):
                position_XLD.append(random.uniform(215, 235))
            # 第14到15个元素的范围是 [215, 248]
            for _ in range(1):
                position_XLD.append(random.uniform(215, 275))
            # 第16到17个元素的范围是 [215, 275]
            for _ in range(2):
                position_XLD.append(random.uniform(215, 275))

            position_XLD = np.insert(position_XLD, 0, self.position_XLD_SD[0])  # 确定初始的和最终的水位
            position_XLD = np.append(position_XLD, self.position_XLD_SD[1])

            Q_agr_values = np.array([np.random.uniform(0.2 * q, q+0.00001) for q in self.Q_agr_list])
            Q_agr = np.array(Q_agr_values)
            # #将两个18维的水位列表和Q _ agr合并为一个54维的列表
            position = np.array(np.concatenate((position_SMX, position_XLD, Q_agr)))
# 生成速度
            vel_SMX = np.array([np.random.uniform(-vel, vel) for vel in vel_inti_SMX]) #决策变量为17个
            vel_XLD = np.array([np.random.uniform(-vel, vel) for vel in vel_inti_XLD]) #决策变量为17个
            vel_arg = np.array([np.random.uniform(-vel, vel) for vel in vel_inti_arg]) #决策变量为18个
            vel=np.array(np.concatenate((vel_SMX, vel_XLD, vel_arg)))
# 初始化
            initial_value=self.objective_function(position,self.W_list, self.In_list_SMX,self.time_list,self.K,self.N,self.interval_flow_list,self.Coefficient,self.Q_life_list,self.Q_agr_list,self.eco_list,self.restraint_list)["D"]
            particle = {
                'position': position,  # #将粒子的位置设置为生成的数组
                "vel":vel,#粒子的速度
                'best_known_position': position,  # 将粒子的最佳已知位置初始化为初始最佳位置
                'best_known_value': initial_value  # 将粒子的最佳已知值初始化为最佳初始值
            }
            particles.append(particle)  # 将粒子添加到粒子列表中

        return particles  # 返回包含所有粒子的列表
# 更新
    def update_particle(self, particle, iteration,temp):
#速度
        c1_max,c1_min = 2,0.9 # 个体学习因子
        c1 = c1_max - (c1_max - c1_min) * iteration / self.num_iterations
        c2_max,c2_min = 2,0.9 #群体学习因子
        c2 = c2_min + (c2_max - c2_min) * iteration / self.num_iterations

        w_max = 0.9 # 权重系数最大值
        w_min = 0.4 # 权重系数最小值

        w = w_max - (w_max - w_min) * iteration/self.num_iterations
# 为三门峡（SMX）更新位置
        particle['vel'][:17] =w* particle['vel'][:17] + c1 * np.random.rand(17) * (particle['best_known_position'][1:18] - particle['position'][1:18]) + c2 * np.random.rand(17) * (self.global_best_position[1:18] -particle['position'][1:18])

        particle['vel'][:17]= np.clip(particle['vel'][:17], -1.3*vel_inti_SMX, 1.3*vel_inti_SMX)

        mean_best_known_SMX_middle = particle['position'][1:18]+particle['vel'][:17]
        new_position_SMX_middle = mean_best_known_SMX_middle
        new_position_SMX = np.insert(new_position_SMX_middle, 0, self.position_SMX_SD[0])  #开头加初水位
        new_position_SMX = np.append(new_position_SMX, self.position_SMX_SD[1])  #结尾加初水位

        new_position_SMX[:11] = np.clip(new_position_SMX[:11], 300, 318)
        new_position_SMX[11:16] = np.clip(new_position_SMX[11:16], 300, 305)
        new_position_SMX[16:] = np.clip(new_position_SMX[16:], 300, 318)

        # 为小浪底（XLD）更新位置

        particle['vel'][17:34] = w*particle['vel'][17:34] + c1 * np.random.rand(17) * (particle['best_known_position'][20:37] - particle['position'][20:37]) + c2 * np.random.rand(17) * (self.global_best_position[20:37] -particle['position'][20:37])

        particle['vel'][17:34]= np.clip(particle['vel'][17:34], -2*vel_inti_XLD, 2*vel_inti_XLD)

        mean_best_known_XLD_middle = particle['position'][20:37]+particle['vel'][17:34]
        new_position_XLD_middle = mean_best_known_XLD_middle
        new_position_XLD = np.insert(new_position_XLD_middle, 0,self.position_XLD_SD[0])   #开头加初水位
        new_position_XLD = np.append(new_position_XLD, self.position_XLD_SD[1])  #结尾加初水位

        new_position_XLD[:10]=np.clip(new_position_XLD[:10], 215, 275)
        new_position_XLD[10:15]=np.clip(new_position_XLD[10:15], 215, 235)
        new_position_XLD[15:17]=np.clip(new_position_XLD[15:17], 215, 254)
        new_position_XLD[17:19]=np.clip(new_position_XLD[17:19], 215, 275)


        #更新Q _ agr的位置

        particle['vel'][34:] =w* particle['vel'][34:] + c1 * np.random.rand(18) * (particle['best_known_position'][38:56] - particle['position'][38:56]) + c2 * np.random.rand(18) * (self.global_best_position[38:56] -particle['position'][38:56])

        particle['vel'][34:]= np.clip(particle['vel'][34:], -1.5*vel_inti_arg, 1.5*vel_inti_arg)

        mean_best_known_Q_agr = particle['position'][38:56]+particle['vel'][34:]
        mean_best_known_Q_agr[2:17] = mean_best_known_Q_agr[2:17]

        arg_max_list=[0.3807261208577,0.7614522417154,116.121466861598,357.121101364522,603.450901559454,603.450901559454,603.450901559454,666.270711500975,666.270711500975,666.270711500975,582.510964912281,582.510964912281,582.510964912281,411.564936647174,168.280945419103,247.091252436647,212.064449317739,0.3807261208577]
        arg_max_list=np.array(arg_max_list)
        arg_min_list=arg_max_list*0.1
        arg_min_list[0]=0.3807261208577
        arg_min_list[1]=0.7614522417154
        arg_min_list[17]=0.3807261208577
        new_Q_agr = np.clip(mean_best_known_Q_agr, arg_min_list, arg_max_list)
#将三个部分结合起来得到新的定位
        new_position = np.concatenate((new_position_SMX, new_position_XLD, new_Q_agr))
#评价
        current_value=self.objective_function(particle['position'],self.W_list, self.In_list_SMX,self.time_list,self.K,self.N,self.interval_flow_list,self.Coefficient,self.Q_life_list,self.Q_agr_list,self.eco_list,self.restraint_list)["D"]
        new_value=self.objective_function(new_position,self.W_list, self.In_list_SMX,self.time_list,self.K,self.N,self.interval_flow_list,self.Coefficient,self.Q_life_list,self.Q_agr_list,self.eco_list,self.restraint_list)["D"]

        if new_value > current_value or np.random.rand() < np.exp(( new_value - current_value  ) / temp):
            return new_position
        else:
            return particle['position']

    def run(self):
        particles = self.initialize_particles()  # 初始化粒子

        for iteration in range(self.num_iterations):
            temp = self.initial_temp * ((self.final_temp / self.initial_temp) ** (iteration / self.num_iterations))  # 计算当前的温度
            print(iteration)
            for particle in particles:
                position = particle['position']

                total_synergy_degree = self.objective_function(position,self.W_list, self.In_list_SMX,self.time_list,self.K,self.N,self.interval_flow_list,self.Coefficient,self.Q_life_list,self.Q_agr_list,self.eco_list,self.restraint_list)["D"] # 计算目标函数值
                if total_synergy_degree > particle['best_known_value']:
                    particle['best_known_position'] = position
                    particle['best_known_value'] = total_synergy_degree

                if total_synergy_degree > self.global_best_value:
                    self.global_best_position = position
                    self.global_best_value = total_synergy_degree
            self.global_best_values.append(self.global_best_value)  # 保存当前迭代的全局最优值
            for particle in particles:
                particle['position'] = self.update_particle(particle, iteration,temp)  # 更新粒子的位置
     # 最后一次迭代结束后，再次调用目标函数，获取所有的返回值
        result = self.objective_function(self.global_best_position,self.W_list, self.In_list_SMX,self.time_list,self.K,self.N,self.interval_flow_list,self.Coefficient,self.Q_life_list,self.Q_agr_list,self.eco_list,self.restraint_list)
        return self.global_best_position, result, self.global_best_values

In [ ]:

Z_list_CHU_1=[317.6063]+[305]*17+[317.6063]+[263.84447]+[250]*17+[256.57211]+[10]*18
initial_result = objective_function(Z_list_CHU,W_list, In_list_SMX,time_list,K,N,interval_flow_list,Coefficient,Q_life_list,Q_agr_list,eco_list,restraint_list)
initial_best_value = initial_result["D"]
initial_best_position=Z_list_CHU
# print(initial_best_value)
# print(Z_list_CHU)

data=initial_result["data"]
data["chu_position_SMX"]=list(Z_list_CHU[:19])
data["chu_position_XLD"]=list(Z_list_CHU[19:38])
data["chu_position_arg"]=list(Z_list_CHU[38:56])
name="平水年优化前"

store_data(data,name)

In [ ]:
print(initial_best_value)

num_particles = 200# 粒子的数量
num_iterations =700# 迭代的次数
initial_temp = 100  # 初始温度
final_temp = 0.1  # 最终温度



sapso_max_SD = SAPSO(objective_function, num_particles, num_iterations, initial_temp, final_temp,position_SMX_SD,position_XLD_SD,W_list, In_list_SMX,time_list,K, N,interval_flow_list,Coefficient,Q_life_list,Q_agr_list,eco_list,restraint_list,initial_best_position,initial_best_value)


best_position,result, global_best_values_SD = sapso_max_SD.run()  # 记录最优位置，最优的结果，历史变化值
# print(best_position,result, global_best_values_SD)
data=result["data"]
data["best_position_SMX"]=list(best_position[:19])
data["chu_position_SMX"]=list(Z_list_CHU[:19])
data["best_position_XLD"]=list(best_position[19:38])
data["chu_position_XLD"]=list(Z_list_CHU[19:38])
data["best_position_arg"]=list(best_position[38:56])
data["chu_position_arg"]=list(Z_list_CHU[38:56])
name=f"调度结果"
store_data(data,name)